# Measuring Exposure at Default


**Import the libraries that will be used**

In [ ]:
import urllib.request as request
import json
from jsonschema import validate
import jsonschema
import pandas as pd
import numpy as np

Use json.loads to convert both the sample data and schema into JSON object

In [ ]:
schema_path = request.urlopen("https://raw.githubusercontent.com/SuadeLabs/fire/master/v1-dev/derivative.json").read()
schema = json.loads(schema_path)
print (schema)

In [ ]:
file_path = request.urlopen("https://raw.githubusercontent.com/mariankhoury/SuadeExercise/main/Data.json").read()
data = json.loads(file_path)
print (data)

Use validate method is run to validate the data and that the data matches the schema

In [ ]:
def validate(data):
    for item in data['data']:
        try:
            jsonschema.validate(item, schema)
        except Exception as e:
            raise e

In [ ]:
validate(data)

Convert the sample data to a dataframe using pandas library

In [ ]:
df = pd.DataFrame(data['data'])

**To calculate the adjusted notional:**

First, use pandas to parse contract dates into datetime objects

In [ ]:
start = pd.to_datetime(df['start_date'].values)
end = pd.to_datetime(df['end_date'].values)
current = pd.to_datetime(df['date'].values)

Calculate both S and E and add their columns to the dataframe 

In [ ]:
df['E'] = (end - start).days / 365

In [ ]:
S = (start-current).days/365
df['S'] = np.where(
    S >0, S, np.where(
    S <  0, 0,-1)) 
#where argument in order to return 0 for any negative numbers

To calculate the **adjusted notional**, calculate the **supervisory duration** first, then multiply by the notional amount. The supervisory duration calculation is referncing **paragraph 157** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
sup_duration = (np.exp(-0.05 * df['S']) - np.exp(-0.05 * df['E'])) / 0.05

**Adjusted notional** calculating references **paragraph 157** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
df['adj_notional']= (df['notional_amount'] * sup_duration).values
#add the adjusted notional to the dataframe

Calculate **replacement cost** as outlined in **paragraph 136** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
def replacement_cost():
    replacement_cost = np.sum(df['mtm_dirty'].values)
    return max(replacement_cost, 0)

In [ ]:
derivatives_values = np.sum(df['mtm_dirty'].values)

Calculate the **supervisory delta adjustments** as in **paragraph 159** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
def floating_fixed(x):
    if x == 'fixed':
        return -1
    else:
        return 1  

In [ ]:
df['supervisory_delta']= df['receive_type'].apply(lambda x: floating_fixed(x)).values

Calculate the **maturity factor** by first calculating the **maturities of the derivatives**, as outlined below. This is referenced in **paragraph 164** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
def swap_maturity():
    end =pd.to_datetime(df['end_date'].values)
    current = pd.to_datetime(df['date'].values)
    swap_maturity = (end - current).days/365
    return swap_maturity.values

In [ ]:
swap_maturity = swap_maturity()

In [ ]:
def maturity_factor():
    mf = list(map(lambda x: min(x,1), swap_maturity))
    return np.sqrt(mf) 

The **effective notional** is calculated based on the derived parameters and is then added to the dataframe, referencing **paragraph 167** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
df['effective_notional']= df['adj_notional'] * df['supervisory_delta'] * maturity_factor()


The **Add-on** is calculated below, according to **paragraphs 166-169** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
def addon():
    addon = np.sum(0.005*df['effective_notional'])
    return addon

In [ ]:
addon = addon()

Finally, in order to calculate the **Exposure at Default**, the multiplier is calculated as outlined in **paragraph 148** of http://www.bis.org/publ/bcbs279.pdf

In [ ]:
floor = 0.0
derivatives_values= np.sum(df['mtm_dirty'].values)
numerator = derivatives_values
denominator = 2 * (1 - floor) * addon
exponent = np.exp(numerator / denominator)


In [ ]:
def multiplier():
    multiplier = floor + (1 - floor) * exponent
    multiplier = min(1, multiplier)
    return multiplier

In [ ]:
multiplier = multiplier()

Now, we have all the variables to calculate the **Exposure at Default** as outlined in **paragraph 128** of http://www.bis.org/publ/bcbs279.pdf. The calculation is:

In [ ]:
rc = replacement_cost()
alpha = 1.4
exposure_at_default = alpha * (rc + (multiplier * addon))

In [ ]:
exposure_at_default